# MLOps: Deploy and Monitor

In [1]:
import os
import wml_sdk_utils as wml_util
import wos_sdk_utils as wos_util
import ws_utils as ws_util

wos_client = wos_util.get_client()

/opt/conda/envs/Python-3.8-CUDA/lib/python3.8/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'internal-nginx-svc'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/opt/conda/envs/Python-3.8-CUDA/lib/python3.8/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'internal-nginx-svc'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


## User Configuration
### deployment admin to decide

In [2]:
# decide which wml space & openscale deployment service provider to use
WML_SPACE_ID = '81c825b4-a1ae-4c1e-8cf3-51b6e3b301b7' 
SERVICE_PROVIDER_NAME = "OpenScale Headless Service Provider"

# infer asset name based on the folder name you will be uploading
# if you received a model/script folder named folderA, then the final asset created in WML space will be folderA.zip
FN_MODEL = 'Test_Model_wendy_ws_serialized.zip'
DEPENDENCY_FILENAME = 'deepliif-base.zip'

# decide as you wish
MODEL_NAME = 'DeepLIIF rich test' # just give it name :)
SUBSCRIPTION_NAME = "DeepLIIF Monitor" # a new subscription name

### input from model owner: deployment related

In [3]:
# model and script files for deployment
PATHS = ['/mnts/AdditionalDeepLIIFVolume/deepliif-ws-output/Test_Model_wendy_ws_serialized2',
         '/userfs/wmla-tutorial/deployment-wmla/edi_deployment_files/deepliif-base'
        ]

# WMLA deployment information
DEPLOYMENT_NAME = 'deepliif-base'
RESOURCE_CONFIGS = {'enable_gpus':'True',
                    'n_cpus':8,
                    'memory_allocation':10000,
                    'n_replicas':1,
                    'n_min_kernels':1,
                    'task_execution_timeout':2*60}

### input from model owner: monitor related

In [4]:
# openscale monitor related
DIR_GT = 'DeepLIIF_Datasets/model_eval/gt_images'
DIR_PRED = 'DeepLIIF_Datasets/model_eval/model_images'
VOLUME_DISPLAY_NAME = 'AdditionalDeepLIIFVolume'

# use the function to get default thresholds if default thresholds are desired
THRESHOLDS_GENERIC = wos_util.get_default_thresholds('generic_metrics',wos_client)

THRESHOLDS_SEGMENTATION = {'precision': {'threshold': [60.0, 'lower']},
 'precision_positive': {'threshold': [0.6, 'lower']},
 'precision_negative': {'threshold': [0.6, 'lower']},
 'recall': {'threshold': [15.0, 'lower']},
 'recall_positive': {'threshold': [0.15, 'lower']},
 'recall_negative': {'threshold': [0.15, 'lower']},
 'f1': {'threshold': [45.0, 'lower']},
 'f1_positive': {'threshold': [0.45, 'lower']},
 'f1_negative': {'threshold': [0.45, 'lower']},
 'Dice': {'threshold': [20.0, 'lower']},
 'Dice_positive': {'threshold': [0.20, 'lower']},
 'Dice_negative': {'threshold': [0.20, 'lower']},
 'IOU': {'threshold': [12.0, 'lower']},
 'IOU_positive': {'threshold': [0.12, 'lower']},
 'IOU_negative': {'threshold': [0.12, 'lower']},
 'PixAcc': {'threshold': [35.0, 'lower']},
 'PixAcc_positive': {'threshold': [0.35, 'lower']},
 'PixAcc_negative': {'threshold': [0.35, 'lower']}}

/opt/conda/envs/Python-3.8-CUDA/lib/python3.8/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'internal-nginx-svc'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


### deploy admin to modify `'openscale_custom_metric_provider'` section 
Make sure you added all the monitors you need / are asked to use.

In [5]:
# use double brackets to indicate that it's not for format to fill in a value but an actual bracket
metadata = """
{{{model_asset_id}:{{
                 'model_asset': {FN_MODEL},
                 'model_name': {MODEL_NAME},
                 'deployment_id': None,
                 'deployment_space_id':{WML_SPACE_ID},
                 'openscale_subscription_id': None,
                 'openscale_custom_metric_provider': 
                     {{'segmentation_metrics':{{'dir_gt': {DIR_GT},
                                              'dir_pred': {DIR_PRED},
                                              'volume_display_name': {VOLUME_DISPLAY_NAME},
                                              'most_recent': 5,
                                              'thresholds': {THRESHOLDS_SEGMENTATION}}},
                      'generic_metrics': {{'dir_gt': {DIR_GT},
                                          'dir_pred': {DIR_PRED},
                                          'volume_display_name': {VOLUME_DISPLAY_NAME},
                                          'most_recent': 1,
                                          'thresholds': {THRESHOLDS_GENERIC}}}
                     }},
                'wmla_deployment':{{'deployment_name':{DEPLOYMENT_NAME},
                                   'deployment_url':{deployment_url},
                                   'dependency_filename':{DEPENDENCY_FILENAME},
                                   'volume_display_name':{VOLUME_DISPLAY_NAME},
                                   'resource_configs':{RESOURCE_CONFIGS}}}
             }}}}
"""

### settings you don't need to worry about

In [6]:
data_mart_id = '00000000-0000-0000-0000-000000000000'
deployment_url = f'https://wmla-inference-cpd-wmla.apps.cpd.mskcc.org/dlim/v1/inference/{DEPLOYMENT_NAME}'

## 1. Publish Assets to WML Deployment Space

In [7]:
wml_client = wml_util.get_client(space_id=WML_SPACE_ID)
wml_client.version

'1.0.141'

In [8]:
# wml_util.list_files(wml_client)

In [9]:
wml_util.upload_batch(PATHS,wml_client,overwrite=True)

Creating data asset...
SUCCESS
Finished publishing /userfs/tmp/Test_Model_wendy_ws_serialized2.zip as Test_Model_wendy_ws_serialized2.zip
Finished deleting existing old assets with the same name
Creating data asset...
SUCCESS
Finished publishing /userfs/tmp/deepliif-base.zip as deepliif-base.zip
Finished deleting existing old assets with the same name


## 2. Update Metadata yaml for Main Model Asset
This yaml file contains deployment information about a corresponding model file, plus needed input for a custom metric provider to use.

In [10]:
def metadata_yml_fill(metadata_str,varnames_additional=[]):
    variables = [x for x in globals() if x.isupper()] + varnames_additional
    d_variables = {k:repr(eval(k)) for k in variables}
    metadata_str_filled = metadata_str.format(**d_variables)
    
    return eval(metadata_str_filled)

In [11]:
data_assets = wml_util.list_files(wml_client,keep_only_latest=True)
model_asset_id = [k for k,v in data_assets.items() if v == FN_MODEL][0]

In [12]:
metadata = metadata_yml_fill(metadata,['model_asset_id','deployment_url'])
# metadata

In [13]:
wml_util.metadata_yml_add(metadata,wml_client,overwrite=True)

Successfully saved data asset content to file: 'tmp/deployment_metadata.yml'
Key d1a14d91-d223-4895-bc4d-4badfbb80172 already exists in yaml file deployment_metadata.yml, updating the values...
Writing new metadata in..
Creating data asset...
SUCCESS
Finished publishing tmp/deployment_metadata.yml as deployment_metadata.yml
Finished deleting existing old assets with the same name


In [14]:
wml_util.metadata_yml_load(wml_client)[model_asset_id]

Successfully saved data asset content to file: 'tmp/deployment_metadata.yml'


{'deployment_id': None,
 'deployment_space_id': '81c825b4-a1ae-4c1e-8cf3-51b6e3b301b7',
 'model_asset': 'Test_Model_wendy_ws_serialized.zip',
 'model_name': 'DeepLIIF rich test',
 'openscale_custom_metric_provider': {'generic_metrics': {'dir_gt': 'DeepLIIF_Datasets/model_eval/gt_images',
   'dir_pred': 'DeepLIIF_Datasets/model_eval/model_images',
   'most_recent': 1,
   'thresholds': {'num_images_recent_ground_truth': {'threshold': [5.0,
      'lower']},
    'num_images_recent_predicted': {'threshold': [40.0, 'lower']},
    'num_images_total_ground_truth': {'threshold': [5.0, 'lower']},
    'num_images_total_predicted': {'threshold': [40.0, 'lower']}},
   'volume_display_name': 'AdditionalDeepLIIFVolume'},
  'segmentation_metrics': {'dir_gt': 'DeepLIIF_Datasets/model_eval/gt_images',
   'dir_pred': 'DeepLIIF_Datasets/model_eval/model_images',
   'most_recent': 5,
   'thresholds': {'Dice': {'threshold': [20.0, 'lower']},
    'Dice_negative': {'threshold': [0.2, 'lower']},
    'Dice_posi

## Kick off Deployment & Monitoring Pipeline

In [15]:
import os
import nbformat
from nbconvert.preprocessors import ExecutePreprocessor
import ws_utils as ws_util

In [16]:
# pass variables needed in pipeline notebooks through env vars
os.environ['SERVICE_PROVIDER_NAME'] = SERVICE_PROVIDER_NAME
os.environ['SUBSCRIPTION_NAME'] = SUBSCRIPTION_NAME

os.environ['MODEL_ASSET_ID'] = model_asset_id
os.environ['WML_SPACE_ID'] = WML_SPACE_ID

os.environ['WOS_GUID'] = data_mart_id

In [17]:
paths_nb = ['A3_OpenScale_Configuration.ipynb']

for path_nb in paths_nb:
    print('-'*30,path_nb,'-'*30)
    ws_util.run_pipeline_notebook(path_nb,save_notebook=True)

------------------------------ A3_OpenScale_Configuration.ipynb ------------------------------
Executed notebook written to A3_OpenScale_Configuration_out.ipynb
Successfully saved data asset content to file: 'tmp/deployment_metadata.yml'

Successfully saved data asset content to file: 'tmp/monitor_metadata.yml'

Service provider ID: 1059fa1d-7dfc-4d7b-8e40-ae8aef72afe1

Subscription ID: 7fc8fa35-85b3-49cf-97ec-15df1e6971ad

Successfully saved data asset content to file: 'tmp/deployment_metadata.yml'
Creating data asset...

SUCCESS
Finished publishing tmp/deployment_metadata.yml as deployment_metadata.yml

Finished deleting existing old assets with the same name

Successfully saved data asset content to file: 'tmp/deployment_metadata.yml'

{'deployment_id': None,
 'deployment_space_id': '81c825b4-a1ae-4c1e-8cf3-51b6e3b301b7',
 'model_asset': 'Test_Model_wendy_ws_serialized.zip',
 'model_name': 'DeepLIIF rich test',
 'openscale_custom_metric_provider': {'generic_metrics': {'dir_gt': 'Dee